# 📘 Transfer Learning com TensorFlow

Este notebook demonstra como aplicar **Transfer Learning** utilizando o modelo **MobileNetV2** pré-treinado no ImageNet.

Dataset utilizado: [Cats vs Dogs](https://www.tensorflow.org/datasets/catalog/cats_vs_dogs).

In [ ]:
!pip install tensorflow matplotlib numpy pandas seaborn scikit-learn

## 1. Importação das bibliotecas

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib

## 2. Carregando o Dataset

No Google Colab, podemos usar `tf.keras.utils.get_file` para baixar o dataset de gatos e cachorros.

In [ ]:
dataset_url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
data_dir = tf.keras.utils.get_file('cats_and_dogs_filtered', origin=dataset_url, extract=True)
data_dir = pathlib.Path(data_dir).with_suffix('')

train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')

## 3. Pré-processamento do Dataset

In [ ]:
batch_size = 32
img_size = (160, 160)

train_dataset = image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=img_size)

val_dataset = image_dataset_from_directory(
    val_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=img_size)

## 4. Configurando o Modelo Pré-treinado (MobileNetV2)

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

IMG_SHAPE = img_size + (3,)
base_model = MobileNetV2(input_shape=IMG_SHAPE,
                         include_top=False,
                         weights='imagenet')

base_model.trainable = False

global_average_layer = layers.GlobalAveragePooling2D()
prediction_layer = layers.Dense(1, activation='sigmoid')

model = models.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

## 5. Treinamento do Modelo

In [ ]:
history = model.fit(train_dataset,
                    epochs=5,
                    validation_data=val_dataset)

## 6. Avaliação do Modelo

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## 7. Salvando o Modelo

In [ ]:
model.save("models/transfer_learning_model.h5")